<h2 style="color:black;">Lab 1 - Scraping and NLP Pipeline</h2>

<hr style="border:2px solid black;">

<h4 style="color:black;">Realised by:</h4>
<ul>
    <li><strong style="color:black;">Aicha Kharbach</strong></li>
</ul>

<h4 style="color:black;">Guided by:</h4> 
<ul>
    <li><strong style="color:black;">Pr . ELAACHAk LOTFI</strong></li>
</ul>
<p><strong>Objective :</strong> The main purpose behind this lab is to get familiar with Scraping and NLP
 Pipeline.</p>


<p>I thoroughly parsed the HTML structure of the 'Fatafeat' culinary website to extract data, including recipes and ingredients, using BeautifulSoup, and then stored it in MongoDB, followed by the application of NLP techniques.</p>


#       Summary

1. [Data Loading](#data-loading)
2. [NLP Pipeline](#nlp-pipeline)
    1. [Text Cleaning](#text-cleaning)
    2. [Tokenization](#tokenization)
    3. [Stop words](#stop-words)
    4. [Discretization](#discretization)
    5. [Normalization](#nor)
    6. [ Stemming and Lemmatization](#SL)
3. [Parts of Speech (POS)](#part)
4. [Named Entity Recognition(NER)](#ner)
5. [Lab Learning Recap](#recap)

## 1. Data Loading <a id='data-loading'></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pymongo
import pandas as pd

In [2]:
def scrape_recipe_data(url):
    titles = []
    properties = []
    chefs = []
    ingredients = []
    recipes = []
    divs = []
    links = []
    
    # Retrieve the content of the web page
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, "lxml")

    # Find elements on the web page containing the data
    titles_elements = soup.find_all("h2", {"class": "mb-0 semibold-font"})
    properties_elements = soup.find_all("div", {"class": "tabkha-details rounded-pill py-1 me-1 medium-font"})
    chefs_elements = soup.find_all("h3", {"class": "pt-5 recipe-chef-name medium-font"})
    div_elements = soup.find_all("a", {"class": "recipe-real-image"})

    # Loop to extract and store the data
    for i in range(len(titles_elements)):
        titles.append(titles_elements[i].text)
        properties.append(properties_elements[i].text)
        divs.append(div_elements[i].text)
        links.append(div_elements[i].attrs['href'])

    # Loop to extract ingredients and recipes from each recipe's page
    for link in links:
        result = requests.get(link)
        src = result.content
        soup = BeautifulSoup(src, "lxml")
        ingredients_elements = soup.find("div", {"class": "row px-0 px-lg-3 ingredients"})
        if ingredients_elements:
            ingredients.append(ingredients_elements.text.strip())
        else:
            ingredients.append("Ingredients not available")
        recipes_elements = soup.find("div", {"class": "row px-0 px-lg-3 preparationWay"})
        if recipes_elements:
            recipes.append(recipes_elements.text.strip())
        else:
            recipes.append("Recipes not available")

    return titles, properties, ingredients, recipes


In [3]:
def save_to_mongodb(data):
    client = pymongo.MongoClient("mongodb://localhost:27017")
    db = client["Atelier"]
    collection = db["scraping"]
    for i in range(len(data[0])):
        # Construct a dictionary for the recipe with corresponding data
        recipe = {
            "title": data[0][i],
            "properties": data[1][i],
            "ingredients": data[2][i],
            "recipes": data[3][i]
        }
        collection.insert_one(recipe)
    client.close()


In [4]:
# Scrape and save cake recipes
cake_data = scrape_recipe_data("https://www.fatafeat.com/recipes-list/207106-%D9%83%D9%8A%D9%83")
save_to_mongodb(cake_data)

# Scrape and save cookies recipes
cookies_data = scrape_recipe_data("https://www.fatafeat.com/recipes-list/207104-%D9%83%D9%88%D9%83%D9%8A%D8%B2")
save_to_mongodb(cookies_data)

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Atelier"]
collection = db["scraping"]

In [5]:
# Fetch data from MongoDB
cursor = collection.find()

In [6]:
# Convert the cursor to a list of dictionaries
data = list(cursor)

In [7]:
# Create a DataFrame from the data
df = pd.DataFrame(data)

In [8]:
# Close the MongoDB connection
client.close()

In [9]:
# Display the DataFrame
df.head(10)

,_id,title,properties,ingredients,recipes
0,66134fc115ff5e1dc26863b7,تشيز كيك قليل الدسم,سهلة,180 \n\n\nجرام بسكويت عادي\n\n\n\n\n\n\n 90 \n...,1\n\nيحمى الفرن إلى 160 درجة مئوية (بدون مروحة...
1,66134fc115ff5e1dc26863b8,كيك الموز بالتوفي,سهلة,1 1/2 \n\n\nكوب دقيق\n\n\n\n\n\n\n 1 \n\n\nملع...,1\n\nيثبت الرف الشبكي في وسط الفرن. يسخن الفرن...
2,66134fc115ff5e1dc26863b9,كيك ماربل بالكاكاو,سهلة,3 \n\n\nكوب دقيق\n\n\n\n\n\n\n 3 \n\n\nملعقة ص...,1\n\nيحضر قالب ذو فتحة من الوسط سعة 12 كوب. يد...
3,66134fc115ff5e1dc26863ba,كيك القرع واللوز,سهلة,125 \n\n\nجرام دقيق اللوز\n\n\n\n\n\n\n 25 \n\...,1\n\nيخلط دقيق اللوز والطحين متعدد الاستعمالات...
4,66134fc115ff5e1dc26863bb,كعكة‭ ‬الجواهر,سهلة,0 \n\n\nعبوة تحتوي على 4 أنواع مختلفة من الجيل...,Recipes not available
5,66134fc115ff5e1dc26863bc,كعكة‭ ‬البلاك‭ ‬فورست,سهلة,200 \n\n\nجم الدقيق\n\n\n\n\n\n\n 50 \n\n\nجم ...,1\n\nتُخفق‭ ‬الزبدة‭ ‬مع‭ ‬السكر‭ ‬حتى‭ ‬يتكون...
6,66134fc115ff5e1dc26863bd,مخبوز البانوفى بالموز ( كيك...,متوسطة,190 \n\n\nجرام دقيق\n\n\n\n\n\n\n 0 \n\n\nملعق...,1\n\nلعمل الصوص : يرفع السكر مع الماء على النا...
7,66134fc115ff5e1dc26863be,كيك‭ ‬التمر,سهلة,2 \n\n\n بيضة\n\n\n\n\n\n\n 3 \n\n\n أكواب من ...,1\n\nتُنخل‭ ‬المكونات‭ ‬الجافة‭ ‬معاً‭ ‬فيما‭ ...
8,66134fc115ff5e1dc26863bf,كيكة السميد,سهلة,1/2 \n\n\nكوب دقيق\n\n\n\n\n\n\n 2 \n\n\nملعقة...,1\n\nيسخن الفرن لدرجة حرارة 180. يحضر قالب مست...
9,66134fc115ff5e1dc26863c0,كاب كيك بشوكولاتة فيريرو,متوسطة,125 \n\n\nجرام زبدة\n\n\n\n\n\n\n 1 1/2 \n\n\n...,1\n\nيخفق البيض والسكر والفانيليا حتى يصبح الم...


## 2. NLP Pipeline <a id='nlp-pipeline'></a>

### 2.1 Text Cleaning <a id='text-cleaning'></a>

In [10]:
import re
def clean_arabic_text(text):
    cleaned_text = re.sub(r'[^\u0621-\u064A\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()
# Apply cleaning to each column 
df['title'] = df['title'].apply(clean_arabic_text)
df['properties'] = df['properties'].apply(clean_arabic_text)
df['ingredients'] = df['ingredients'].apply(clean_arabic_text)
df['recipes'] = df['recipes'].apply(clean_arabic_text)

In [11]:
# Display the DataFrame
df.head(10)

,_id,title,properties,ingredients,recipes
0,66134fc115ff5e1dc26863b7,تشيز كيك قليل الدسم,سهلة,جرام بسكويت عادي جرام زبدة جرام جبنة الكريم جر...,يحمى الفرن إلى درجة مئوية بدون مروحة تغلف قاعد...
1,66134fc115ff5e1dc26863b8,كيك الموز بالتوفي,سهلة,كوب دقيق ملعقة كبيرة بيكنج باودر ملعقة صغيرة م...,يثبت الرف الشبكي في وسط الفرن يسخن الفرن إلى د...
2,66134fc115ff5e1dc26863b9,كيك ماربل بالكاكاو,سهلة,كوب دقيق ملعقة صغيرة بيكنج باودر ملعقة صغيرة م...,يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...
3,66134fc115ff5e1dc26863ba,كيك القرع واللوز,سهلة,جرام دقيق اللوز جرام طحين متعدد الاستعمالات مل...,يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...
4,66134fc115ff5e1dc26863bb,كعكة الجواهر,سهلة,عبوة تحتوي على أنواع مختلفة من الجيلاتين المنك...,
5,66134fc115ff5e1dc26863bc,كعكة البلاك فورست,سهلة,جم الدقيق جم الكاكاو البودرة ملعقة صغيرة البيك...,تخفق الزبدة مع السكر حتى يتكون خليطا خفيفا وزغ...
6,66134fc115ff5e1dc26863bd,مخبوز البانوفى بالموز كيك,متوسطة,جرام دقيق ملعقة صغيرة و نصف بيكنج باودر جرام ز...,لعمل الصوص يرفع السكر مع الماء على النار درجة ...
7,66134fc115ff5e1dc26863be,كيك التمر,سهلة,بيضة أكواب من الدقيق ملعقة صغيرة برش الليمون م...,تنخل المكونات الجافة معا فيما عدا السكر توضع ب...
8,66134fc115ff5e1dc26863bf,كيكة السميد,سهلة,كوب دقيق ملعقة صغيرة بيكنج باودر كوب سميد ناعم...,يسخن الفرن لدرجة حرارة يحضر قالب مستدير مقاس ب...
9,66134fc115ff5e1dc26863c0,كاب كيك بشوكولاتة فيريرو,متوسطة,جرام زبدة كوب سكر ملعقة صغيرة فانيليا حبة بيض ...,يخفق البيض والسكر والفانيليا حتى يصبح المزيج ب...


### 2.2 Tokenization <a id='tokenization'></a>

In [12]:
from nltk.tokenize import WordPunctTokenizer
# Create an instance of the WordPunctTokenizer
tokenizer = WordPunctTokenizer()

def tokenize_text(text):
    return tokenizer.tokenize(text)

df['ingredients_tokenized'] = df['ingredients'].apply(tokenize_text)
df['recipes_tokenized'] = df['recipes'].apply(tokenize_text)

print(df[['ingredients_tokenized','recipes_tokenized']].head())

                               ingredients_tokenized  \
0  [جرام, بسكويت, عادي, جرام, زبدة, جرام, جبنة, ا...   
1  [كوب, دقيق, ملعقة, كبيرة, بيكنج, باودر, ملعقة,...   
2  [كوب, دقيق, ملعقة, صغيرة, بيكنج, باودر, ملعقة,...   
3  [جرام, دقيق, اللوز, جرام, طحين, متعدد, الاستعم...   
4  [عبوة, تحتوي, على, أنواع, مختلفة, من, الجيلاتي...   

                                   recipes_tokenized  
0  [يحمى, الفرن, إلى, درجة, مئوية, بدون, مروحة, ت...  
1  [يثبت, الرف, الشبكي, في, وسط, الفرن, يسخن, الف...  
2  [يحضر, قالب, ذو, فتحة, من, الوسط, سعة, كوب, يد...  
3  [يخلط, دقيق, اللوز, والطحين, متعدد, الاستعمالا...  
4                                                 []  


### 2.3 Stop words <a id='stop-words'></a>

In [13]:
from nltk.corpus import stopwords

def remove_stopwords(text, stop_words):
    word_tokens = text.split()
    # Filter out any words that are in the list of stopwords
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    return ' '.join(filtered_sentence)

# Create a set of Arabic stopwords from the NLTK corpus
ar_stopwords = set(stopwords.words('arabic'))
df['recipes_without_stopwords'] = df['recipes'].apply(lambda x: remove_stopwords(x, ar_stopwords))
print(df[['recipes', 'recipes_without_stopwords']].head(10))

                                             recipes  \
0  يحمى الفرن إلى درجة مئوية بدون مروحة تغلف قاعد...   
1  يثبت الرف الشبكي في وسط الفرن يسخن الفرن إلى د...   
2  يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...   
3  يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...   
4                                                      
5  تخفق الزبدة مع السكر حتى يتكون خليطا خفيفا وزغ...   
6  لعمل الصوص يرفع السكر مع الماء على النار درجة ...   
7  تنخل المكونات الجافة معا فيما عدا السكر توضع ب...   
8  يسخن الفرن لدرجة حرارة يحضر قالب مستدير مقاس ب...   
9  يخفق البيض والسكر والفانيليا حتى يصبح المزيج ب...   

                           recipes_without_stopwords  
0  يحمى الفرن درجة مئوية بدون مروحة تغلف قاعدة قا...  
1  يثبت الرف الشبكي وسط الفرن يسخن الفرن درجة حرا...  
2  يحضر قالب فتحة الوسط سعة كوب يدهن القالب بقليل...  
3  يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...  
4                                                     
5  تخفق الزبدة السكر يتكون خليطا خفيفا وزغبا يضاف... 

### 2.4 Discretization <a id='discretization'></a>

In [14]:
# Define bins for categorizing the length of titles
bins = [0, 10, 20, 30] 
labels = ['court', 'moyen', 'long']

df['title_length'] = df['title'].apply(len) 

df['title_length_category'] = pd.cut(df['title_length'], bins=bins, labels=labels)

print(df[['title', 'title_length', 'title_length_category']].head(10))

                       title  title_length title_length_category
0        تشيز كيك قليل الدسم            19                 moyen
1          كيك الموز بالتوفي            17                 moyen
2         كيك ماربل بالكاكاو            18                 moyen
3           كيك القرع واللوز            16                 moyen
4               كعكة الجواهر            12                 moyen
5          كعكة البلاك فورست            17                 moyen
6  مخبوز البانوفى بالموز كيك            25                  long
7                  كيك التمر             9                 court
8                كيكة السميد            11                 moyen
9   كاب كيك بشوكولاتة فيريرو            24                  long


### 2.5 Normalization <a id='nor'></a>

In [15]:
# Normalisation du texte
def normalize_arabic_text(text):
    # Remove Arabic diacritics
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    # Normalize different forms of the letter Alif such as 'أ', 'إ', and 'آ' to a plain 'ا'
    text = re.sub(r'أ|إ|آ', 'ا', text)
    return text

df['recipes_norm'] = df['recipes'].apply(normalize_arabic_text)
print(df[['recipes', 'recipes_norm']].head(10))

                                             recipes  \
0  يحمى الفرن إلى درجة مئوية بدون مروحة تغلف قاعد...   
1  يثبت الرف الشبكي في وسط الفرن يسخن الفرن إلى د...   
2  يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...   
3  يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...   
4                                                      
5  تخفق الزبدة مع السكر حتى يتكون خليطا خفيفا وزغ...   
6  لعمل الصوص يرفع السكر مع الماء على النار درجة ...   
7  تنخل المكونات الجافة معا فيما عدا السكر توضع ب...   
8  يسخن الفرن لدرجة حرارة يحضر قالب مستدير مقاس ب...   
9  يخفق البيض والسكر والفانيليا حتى يصبح المزيج ب...   

                                        recipes_norm  
0  يحمى الفرن الى درجة مئوية بدون مروحة تغلف قاعد...  
1  يثبت الرف الشبكي في وسط الفرن يسخن الفرن الى د...  
2  يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...  
3  يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...  
4                                                     
5  تخفق الزبدة مع السكر حتى يتكون خليطا خفيفا وزغ... 

### 2.6  Stemming and Lemmatization <a id='SL'></a>

In [16]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
# Download NLTK data necessary for tokenization and stemming
nltk.download('wordnet')
nltk.download('punkt')
# Create a set of Arabic stopwords from the NLTK corpus
arabic_stop_words = set(stopwords.words('arabic'))
porter = PorterStemmer()

def stem(text):
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in arabic_stop_words]
    stemmed_words = [porter.stem(w) for w in filtered_sentence]
    return stemmed_words

df['recipes_stemmed'] = df['recipes'].apply(stem)

print(df['recipes_stemmed'].head(10))

0    [يحمى, الفرن, درجة, مئوية, بدون, مروحة, تغلف, ...
1    [يثبت, الرف, الشبكي, وسط, الفرن, يسخن, الفرن, ...
2    [يحضر, قالب, فتحة, الوسط, سعة, كوب, يدهن, القا...
3    [يخلط, دقيق, اللوز, والطحين, متعدد, الاستعمالا...
4                                                   []
5    [تخفق, الزبدة, السكر, يتكون, خليطا, خفيفا, وزغ...
6    [لعمل, الصوص, يرفع, السكر, الماء, النار, درجة,...
7    [تنخل, المكونات, الجافة, معا, السكر, توضع, بقي...
8    [يسخن, الفرن, لدرجة, حرارة, يحضر, قالب, مستدير...
9    [يخفق, البيض, والسكر, والفانيليا, يصبح, المزيج...
Name: recipes_stemmed, dtype: object


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
# Initialize an instance of WordNetLemmatizer for lemmatizing words
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in arabic_stop_words]
    # Lemmatize each word in the filtered sentence
    lemmatized_words = [lemmatizer.lemmatize(w) for w in filtered_sentence]
    return lemmatized_words

df['recipes_lemmatized'] = df['recipes'].apply(lemmatize)

print(df['recipes_lemmatized'].head(10))


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    [يحمى, الفرن, درجة, مئوية, بدون, مروحة, تغلف, ...
1    [يثبت, الرف, الشبكي, وسط, الفرن, يسخن, الفرن, ...
2    [يحضر, قالب, فتحة, الوسط, سعة, كوب, يدهن, القا...
3    [يخلط, دقيق, اللوز, والطحين, متعدد, الاستعمالا...
4                                                   []
5    [تخفق, الزبدة, السكر, يتكون, خليطا, خفيفا, وزغ...
6    [لعمل, الصوص, يرفع, السكر, الماء, النار, درجة,...
7    [تنخل, المكونات, الجافة, معا, السكر, توضع, بقي...
8    [يسخن, الفرن, لدرجة, حرارة, يحضر, قالب, مستدير...
9    [يخفق, البيض, والسكر, والفانيليا, يصبح, المزيج...
Name: recipes_lemmatized, dtype: object


In [18]:
df[['recipes', 'recipes_stemmed', 'recipes_lemmatized']].head(10)

,recipes,recipes_stemmed,recipes_lemmatized
0,يحمى الفرن إلى درجة مئوية بدون مروحة تغلف قاعد...,"[يحمى, الفرن, درجة, مئوية, بدون, مروحة, تغلف, ...","[يحمى, الفرن, درجة, مئوية, بدون, مروحة, تغلف, ..."
1,يثبت الرف الشبكي في وسط الفرن يسخن الفرن إلى د...,"[يثبت, الرف, الشبكي, وسط, الفرن, يسخن, الفرن, ...","[يثبت, الرف, الشبكي, وسط, الفرن, يسخن, الفرن, ..."
2,يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...,"[يحضر, قالب, فتحة, الوسط, سعة, كوب, يدهن, القا...","[يحضر, قالب, فتحة, الوسط, سعة, كوب, يدهن, القا..."
3,يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...,"[يخلط, دقيق, اللوز, والطحين, متعدد, الاستعمالا...","[يخلط, دقيق, اللوز, والطحين, متعدد, الاستعمالا..."
4,,[],[]
5,تخفق الزبدة مع السكر حتى يتكون خليطا خفيفا وزغ...,"[تخفق, الزبدة, السكر, يتكون, خليطا, خفيفا, وزغ...","[تخفق, الزبدة, السكر, يتكون, خليطا, خفيفا, وزغ..."
6,لعمل الصوص يرفع السكر مع الماء على النار درجة ...,"[لعمل, الصوص, يرفع, السكر, الماء, النار, درجة,...","[لعمل, الصوص, يرفع, السكر, الماء, النار, درجة,..."
7,تنخل المكونات الجافة معا فيما عدا السكر توضع ب...,"[تنخل, المكونات, الجافة, معا, السكر, توضع, بقي...","[تنخل, المكونات, الجافة, معا, السكر, توضع, بقي..."
8,يسخن الفرن لدرجة حرارة يحضر قالب مستدير مقاس ب...,"[يسخن, الفرن, لدرجة, حرارة, يحضر, قالب, مستدير...","[يسخن, الفرن, لدرجة, حرارة, يحضر, قالب, مستدير..."
9,يخفق البيض والسكر والفانيليا حتى يصبح المزيج ب...,"[يخفق, البيض, والسكر, والفانيليا, يصبح, المزيج...","[يخفق, البيض, والسكر, والفانيليا, يصبح, المزيج..."


## 3. Parts of Speech (POS) <a id='part'></a>

In [19]:
import stanza
# Initialize the stanza pipeline for Arabic with tokenization and POS tagging processors
nlp = stanza.Pipeline(lang='ar', processors='tokenize,pos')

def pos_tag_arabic(tokens):
    doc = nlp(' '.join(tokens))
    return [(word.text, word.pos) for sent in doc.sentences for word in sent.words]

df['recipes_pos_tags'] = df['recipes_tokenized'].apply(pos_tag_arabic)

print(df[['recipes', 'recipes_pos_tags']].head(10))

2024-04-08 02:30:31 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-08 02:30:32 INFO: Downloaded file to C:\Users\hp\stanza_resources\resources.json
2024-04-08 02:30:32 WARNING: Language ar package default expects mwt, which has been added
2024-04-08 02:30:33 INFO: Loading these models for language: ar (Arabic):
| Processor | Package     |
---------------------------
| tokenize  | padt        |
| mwt       | padt        |
| pos       | padt_charlm |

2024-04-08 02:30:33 INFO: Using device: cpu
2024-04-08 02:30:33 INFO: Loading: tokenize
2024-04-08 02:30:34 INFO: Loading: mwt
2024-04-08 02:30:34 INFO: Loading: pos
2024-04-08 02:30:34 INFO: Done loading processors!


                                             recipes  \
0  يحمى الفرن إلى درجة مئوية بدون مروحة تغلف قاعد...   
1  يثبت الرف الشبكي في وسط الفرن يسخن الفرن إلى د...   
2  يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...   
3  يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...   
4                                                      
5  تخفق الزبدة مع السكر حتى يتكون خليطا خفيفا وزغ...   
6  لعمل الصوص يرفع السكر مع الماء على النار درجة ...   
7  تنخل المكونات الجافة معا فيما عدا السكر توضع ب...   
8  يسخن الفرن لدرجة حرارة يحضر قالب مستدير مقاس ب...   
9  يخفق البيض والسكر والفانيليا حتى يصبح المزيج ب...   

                                    recipes_pos_tags  
0  [(يحمى, VERB), (الفرن, NOUN), (إلى, ADP), (درج...  
1  [(يثبت, VERB), (الرف, NOUN), (الشبكي, ADJ), (ف...  
2  [(يحضر, VERB), (قالب, NOUN), (ذو, DET), (فتحة,...  
3  [(يخلط, VERB), (دقيق, NOUN), (اللوز, NOUN), (و...  
4                                                 []  
5  [(تخفق, VERB), (الزبدة, NOUN), (مع, ADP), (الس... 

## 4. Named Entity Recognition(NER) <a id='ner'></a>

In [25]:
import spacy
# Load the spaCy model for named entity recognition
# The "xx_ent_wiki_sm" is a small multilingual model trained on Wikipedia
nlp = spacy.load("xx_ent_wiki_sm")
def apply_ner(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]
df['recipes_ner'] = df['recipes'].apply(apply_ner)
print(df[['recipes', 'recipes_ner']].head(10))

                                             recipes  \
0  يحمى الفرن إلى درجة مئوية بدون مروحة تغلف قاعد...   
1  يثبت الرف الشبكي في وسط الفرن يسخن الفرن إلى د...   
2  يحضر قالب ذو فتحة من الوسط سعة كوب يدهن القالب...   
3  يخلط دقيق اللوز والطحين متعدد الاستعمالات والب...   
4                                                      
5  تخفق الزبدة مع السكر حتى يتكون خليطا خفيفا وزغ...   
6  لعمل الصوص يرفع السكر مع الماء على النار درجة ...   
7  تنخل المكونات الجافة معا فيما عدا السكر توضع ب...   
8  يسخن الفرن لدرجة حرارة يحضر قالب مستدير مقاس ب...   
9  يخفق البيض والسكر والفانيليا حتى يصبح المزيج ب...   

                                         recipes_ner  
0             [(حتى, ORG), (دقائق, LOC), (حتى, ORG)]  
1                       [(نظيفا, ORG), (اللون, LOC)]  
2   [(دقائق, LOC), (نظيفا, ORG), (وتجف الصلصة, PER)]  
3                           [(حتى, ORG), (حتى, ORG)]  
4                                                 []  
5                       [(بصب قطر كرز مارشينو, PER)] 

## 5. Lab Learning Recap <a id='recap'></a>

<p>During this lab, I acquired practical skills in web scraping and Natural Language Processing (NLP). 
 
Using BeautifulSoup, I learned how to efficiently extract Arabic content from websites and understood the intricacies of storing scraped data in MongoDB. 

The NLP section introduced me to text pre-processing techniques, including tokenization, stop word, stemming, and lemmatization, highlighting their significance in simplifying text analysis. I also explored POS tagging and NER, which taught me how to algorithmically discern and extract the syntactic roles of words and key information from texts.</p>